In [2]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage

In [3]:
agent = create_agent(
    model="gpt-4o-mini",
    checkpointer = InMemorySaver(),
    middleware = [
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger = ("messages", 10)
        )
    ]
)

In [10]:
config = {"configurable": {
    "thread_id": "test1"
    }    
}

In [12]:
# Alternative test data
questions = [
    "What is 2+2?",
    "What is 10*5?",
    "What is 100/4?",
    "What is 15-7?",
    "What is 3*3?",
    "What is 4*4?",
]

for q in questions:
    response=agent.invoke({"messages":[HumanMessage(content=q)]},config)
    print(f"Messages: {response}")
    print(f"Messages: {len(response['messages'])}")


Messages: {'messages': [HumanMessage(content='What is 2+2?', additional_kwargs={}, response_metadata={}, id='2e5c7ffe-0c61-4c99-a822-418cc3f77f62'), AIMessage(content='2 + 2 equals 4.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-Ctp1TAXoWHi5fVzR4Onkzs71v0HaQ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b8274-24b0-7ef0-8563-56ef403149b1-0', usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanM

In [29]:
from langchain.agents.middleware import HumanInTheLoopMiddleware

def read_email_tool(email_id: str) -> str:
    """Mock function to read an email by its ID."""
    return f"Email content for ID: {email_id}"

def send_email_tool(recipient: str, subject: str, body: str) -> str:
    """Mock function to send an email."""
    return f"Email sent to {recipient} with subject '{subject}'"

agent = create_agent(
    model="gpt-4o-mini",
    checkpointer=InMemorySaver(),
    tools = [read_email_tool, send_email_tool],
    middleware=[HumanInTheLoopMiddleware(
        interrupt_on={
            "send_email_tool" : {
                "allowed_decisions" : ["approve", "reject"]
            },
            "read_email_tool" : False
        }
    )]
)

In [30]:
config = {
    "configurable" : {
        "thread_id" : "test_approval"
    }
}

In [31]:
result = agent.invoke({
    "messages" : [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?")]
}, config=config)

In [32]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?", additional_kwargs={}, response_metadata={}, id='4b7595f0-eab3-4ad7-b2a2-b06052025c44'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 97, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CtpJunyffpo7qIEXDPLYytmc0yXW0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8285-9751-7190-aa26-87b36548e9da-0', tool_calls=[{'name': 'send_email_tool', 'args': {'recipient': 'john@test.com', 'subject': 'Hello', 'body': 'How are you?'}, 'id': 'call_aueS52OsnN9F

In [33]:
from langgraph.types import Command

if "__interrupt__" in result:
    result = agent.invoke(
        Command(
            resume={
                "decisions": [
                   { "type" : "approve"}
                ]
            }
        ),
        config= config
    )

In [34]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?", additional_kwargs={}, response_metadata={}, id='4b7595f0-eab3-4ad7-b2a2-b06052025c44'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 97, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CtpJunyffpo7qIEXDPLYytmc0yXW0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8285-9751-7190-aa26-87b36548e9da-0', tool_calls=[{'name': 'send_email_tool', 'args': {'recipient': 'john@test.com', 'subject': 'Hello', 'body': 'How are you?'}, 'id': 'call_aueS52OsnN9F

In [35]:
result = agent.invoke({
    "messages" : [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?")]
}, config=config)

In [36]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?", additional_kwargs={}, response_metadata={}, id='4b7595f0-eab3-4ad7-b2a2-b06052025c44'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 97, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CtpJunyffpo7qIEXDPLYytmc0yXW0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8285-9751-7190-aa26-87b36548e9da-0', tool_calls=[{'name': 'send_email_tool', 'args': {'recipient': 'john@test.com', 'subject': 'Hello', 'body': 'How are you?'}, 'id': 'call_aueS52OsnN9F

In [37]:
from langgraph.types import Command

if "__interrupt__" in result:
    result = agent.invoke(
        Command(
            resume={
                "decisions": [
                   { "type" : "reject"}
                ]
            }
        ),
        config= config
    )

In [38]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject 'Hello' and body 'How are you?", additional_kwargs={}, response_metadata={}, id='4b7595f0-eab3-4ad7-b2a2-b06052025c44'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 97, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CtpJunyffpo7qIEXDPLYytmc0yXW0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8285-9751-7190-aa26-87b36548e9da-0', tool_calls=[{'name': 'send_email_tool', 'args': {'recipient': 'john@test.com', 'subject': 'Hello', 'body': 'How are you?'}, 'id': 'call_aueS52OsnN9F